In [ ]:
import requests
import pandas as pd
from time import sleep
from datetime import datetime
import matplotlib.pyplot as plt
import os

# ---------- STEP 1: Initialization ---------- #

# Load existing data if available
try:
    df = pd.read_csv("crypto_data.csv")
    print(f"📂 Loaded existing dataset with {len(df)} rows.")
except FileNotFoundError:
    df = pd.DataFrame()
    print("🆕 Starting fresh — no existing CSV found.")

# Ensure folder for charts
if not os.path.exists("charts"):
    os.makedirs("charts")


# ---------- STEP 2: API Data Fetcher ---------- #
def api_runner():
    """
    Fetch latest cryptocurrency listings from CoinMarketCap and append to a global DataFrame.
    """
    global df

    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {'start': '1', 'limit': '15', 'convert': 'INR'}
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': '7290f52a0e9c4c838997cc4a5451560c'
    }

    try:
        response = requests.get(url, headers=headers, params=parameters)
        response.raise_for_status()
        data = response.json()

        if 'data' not in data:
            print("⚠️ Warning: 'data' key missing in API response.")
            return

        df_new = pd.json_normalize(data['data'])
        df_new['timestamp'] = datetime.now()

        # Append new data to main DataFrame
        df = pd.concat([df, df_new], ignore_index=True)
        df.to_csv('crypto_data.csv', index=False)

        print(f"✅ [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Data fetched and saved.")

    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
    except Exception as e:
        print(f"⚠️ Unexpected error: {e}")


# ---------- STEP 3: Data Transformation ---------- #
def transform_data():
    """
    Transform and summarize crypto data for insights.
    """
    if df.empty:
        print("⚠️ No data to transform yet.")
        return None, None, None

    print("\n📊 Transforming and analyzing data...")

    # Select important columns
    cols = ['name', 'symbol', 'quote.INR.price', 'quote.INR.percent_change_24h', 'quote.INR.market_cap']
    if not all(col in df.columns for col in cols):
        print("⚠️ Missing columns in data. Transformation skipped.")
        return None, None, None

    # Keep only latest unique entries
    df_clean = df[cols + ['timestamp']].dropna().drop_duplicates(subset='symbol', keep='last')

    # Top 5 gainers and losers
    top_gainers = df_clean.sort_values(by='quote.INR.percent_change_24h', ascending=False).head(5)
    top_losers = df_clean.sort_values(by='quote.INR.percent_change_24h', ascending=True).head(5)

    print("\n🚀 Top 5 Gainers (24h):")
    print(top_gainers[['name', 'symbol', 'quote.INR.percent_change_24h']])

    print("\n📉 Top 5 Losers (24h):")
    print(top_losers[['name', 'symbol', 'quote.INR.percent_change_24h']])

    return df_clean, top_gainers, top_losers


# ---------- STEP 4: Visualization + Save Charts ---------- #
def visualize_data(df_clean, top_gainers, top_losers):
    """
    Generate visualizations and save them as PNG files.
    """
    timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    print("\n📈 Generating and saving visualizations...")

    plt.style.use('seaborn-v0_8-darkgrid')

    # --- Chart 1: Market Cap vs Price ---
    plt.figure(figsize=(10, 6))
    plt.scatter(df_clean['quote.INR.market_cap'], df_clean['quote.INR.price'], alpha=0.7)
    plt.xscale('log')
    plt.yscale('log')
    plt.title('Market Cap vs Price (INR)')
    plt.xlabel('Market Cap (log scale)')
    plt.ylabel('Price (log scale)')
    plt.tight_layout()
    plt.savefig(f'charts/marketcap_vs_price_{timestamp_str}.png')
    plt.close()

    # --- Chart 2: Top 5 Gainers ---
    plt.figure(figsize=(8, 5))
    plt.bar(top_gainers['symbol'], top_gainers['quote.INR.percent_change_24h'], color='green')
    plt.title('Top 5 Gainers (24h)')
    plt.ylabel('Percent Change (24h)')
    plt.xlabel('Symbol')
    plt.tight_layout()
    plt.savefig(f'charts/top_gainers_{timestamp_str}.png')
    plt.close()

    # --- Chart 3: Top 5 Losers ---
    plt.figure(figsize=(8, 5))
    plt.bar(top_losers['symbol'], top_losers['quote.INR.percent_change_24h'], color='red')
    plt.title('Top 5 Losers (24h)')
    plt.ylabel('Percent Change (24h)')
    plt.xlabel('Symbol')
    plt.tight_layout()
    plt.savefig(f'charts/top_losers_{timestamp_str}.png')
    plt.close()

    print(f"✅ Charts saved in 'charts/' folder ({timestamp_str}).")


# ---------- STEP 5: Continuous Execution Loop ---------- #
print("\n🚀 Starting continuous data extraction (1-minute intervals)...")
print("Press Ctrl + C to stop.\n")

try:
    while True:
        api_runner()
        df_clean, gainers, losers = transform_data()
        if df_clean is not None:
            visualize_data(df_clean, gainers, losers)
        print("⏸ Sleeping for 60 seconds...\n")
        sleep(60)

except KeyboardInterrupt:
    print("\n🛑 Process stopped by user. Data saved to 'crypto_data.csv'.")


📂 Loaded existing dataset with 105 rows.

🚀 Starting continuous data extraction (1-minute intervals)...
Press Ctrl + C to stop.

✅ [2025-10-28 18:00:53] Data fetched and saved.

📊 Transforming and analyzing data...

🚀 Top 5 Gainers (24h):
            name symbol  quote.INR.percent_change_24h
115  Hyperliquid   HYPE                      1.006165
118      Stellar    XLM                      0.982930
108          XRP    XRP                      0.800468
111         USDC   USDC                      0.015044
107  Tether USDt   USDT                      0.006758

📉 Top 5 Losers (24h):
             name symbol  quote.INR.percent_change_24h
109           BNB    BNB                     -3.254870
114       Cardano    ADA                     -2.168981
112      Dogecoin   DOGE                     -1.628824
117  Bitcoin Cash    BCH                     -1.572798
106      Ethereum    ETH                     -1.186377

📈 Generating and saving visualizations...
✅ Charts saved in 'charts/' folder (20251